In [1]:
import copy 
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from lib import *
%matplotlib inline
%load_ext Cython

In [12]:
q = 20
N = 10
h = np.zeros(q)
J = np.zeros((N-1, q, q))

In [3]:
def energy_fast(s, h, J):
    energy = np.sum(h[s])
    for i in range(N):
        for j in range(i, N):
            energy += J[j-i-1, s[i], s[j]]
    return energy

In [4]:
%%timeit
s = np.random.randint(q, size=N)
energy_fast(s, h, J)

41 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
%%cython
cimport numpy as np
def energy_cython(np.ndarray[long, ndim=1] s, np.ndarray[double, ndim=1] h, np.ndarray[double, ndim=3] J):
    cdef int N = len(s)
    cdef double energy = 0.0
    cdef int i, j
    for i in range(N):
        energy += h[s[i]]
        for j in range(i, N):
            energy += J[j-i-1, s[i], s[j]]
    return energy

In [22]:
%%timeit
s = np.random.randint(q, size=N)
energy_cython(s, h, J)

2.91 µs ± 38.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
aas_arr = np.array(list(aminoacids))
h = dict(zip(aminoacids, np.zeros(q)))
J0 = np.zeros((len(aminoacids), len(aminoacids)))
J0 = pd.DataFrame(np.asarray(J0), index=list(aminoacids), columns=list(aminoacids)).to_dict()
Jk = [J0]
for gap in range(1, N):
    Jk.append(copy.deepcopy(J0))


In [9]:
%%timeit
s = ''.join(np.random.choice(aas_arr, size=N))
energy_ising(s, h, Jk)

33.7 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
